In [1]:
import sqlalchemy
import pymysql
import pandas as pd
import requests

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
class Config:
    MYSQL_HOST = os.getenv("MYSQL_HOST")
    MYSQL_PORT = int(os.getenv("MYSQL_PORT"))
    MYSQL_USER = os.getenv("MYSQL_USER")
    MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
    MYSQL_DB = os.getenv("MYSQL_DB")
    MYSQL_CHARSET = os.getenv("MYSQL_CHARSET")

In [4]:
engine = sqlalchemy.create_engine(
    "mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(
        user=Config.MYSQL_USER,
        password=Config.MYSQL_PASSWORD,
        host=Config.MYSQL_HOST,
        port=Config.MYSQL_PORT,
        db=Config.MYSQL_DB,
    )
)

In [5]:
# show tables in the database
with engine.connect() as connection:
    result = connection.execute(sqlalchemy.text(f"show tables;")).fetchall()

result

[('customer',), ('product',), ('transaction',)]

In [6]:
# show schema  using the SQL command describe table 
with engine.connect() as connection:
    desc_transaction = connection.execute(sqlalchemy.text(f"describe transaction")).fetchall()
    desc_customer = connection.execute(sqlalchemy.text(f"describe customer")).fetchall()
    desc_product = connection.execute(sqlalchemy.text(f"describe product")).fetchall()

print("=== transaction table === ")
print(desc_transaction)
print("\n=== customer table === ")
print(desc_customer)
print("\n=== product table === ")
print(desc_product)

=== transaction table === 
[('TransactionNo', 'text', 'YES', '', None, ''), ('Date', 'datetime', 'YES', '', None, ''), ('ProductNo', 'text', 'YES', '', None, ''), ('Price', 'double', 'YES', '', None, ''), ('Quantity', 'bigint(20)', 'YES', '', None, ''), ('CustomerNo', 'double', 'YES', '', None, '')]

=== customer table === 
[('CustomerNo', 'double', 'YES', '', None, ''), ('Country', 'text', 'YES', '', None, ''), ('Name', 'text', 'YES', '', None, '')]

=== product table === 
[('ProductNo', 'text', 'YES', '', None, ''), ('ProductName', 'text', 'YES', '', None, '')]


In [7]:
# query table

customer = pd.read_sql("SELECT * FROM customer", engine)
customer

,CustomerNo,Country,Name
0,17490.0,United Kingdom,Sara Griffin
1,13069.0,United Kingdom,Michael Holt
2,12433.0,Norway,Kelli Sandoval
3,13426.0,United Kingdom,Dalton Graves
4,17364.0,United Kingdom,Michelle James
...,...,...,...
4734,16274.0,United Kingdom,Megan Young
4735,14142.0,United Kingdom,Luke Williams
4736,13065.0,United Kingdom,Lisa Jones
4737,18011.0,United Kingdom,Kelly Jenkins


In [8]:
product = pd.read_sql("SELECT * FROM product", engine)
product

,ProductNo,ProductName
0,22485,Set Of 2 Wooden Market Crates
1,22596,Christmas Star Wish List Chalkboard
2,23235,Storage Tin Vintage Leaf
3,23272,Tree T-Light Holder Willie Winkie
4,23239,Set Of 4 Knick Knack Tins Poppies
...,...,...
3763,22275,Weekend Bag Vintage Rose Paisley
3764,16161M,Wrap Pink Flock
3765,84854,Girly Pink Tool Set
3766,82615,Pink Marshmallow Scarf Knitting Kit


In [9]:
transaction = pd.read_sql("SELECT * FROM transaction", engine)
transaction

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo
0,581482,2024-05-10,22485,21.47,12,17490.0
1,581475,2024-05-10,22596,10.65,36,13069.0
2,581475,2024-05-10,23235,11.53,12,13069.0
3,581475,2024-05-10,23272,10.65,12,13069.0
4,581475,2024-05-10,23239,11.94,6,13069.0
...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0
536346,C536548,2023-05-03,21218,14.09,-3,12472.0
536347,C536548,2023-05-03,20957,11.74,-1,12472.0
536348,C536548,2023-05-03,22580,16.35,-4,12472.0


In [10]:
merged_transaction = transaction.merge(product, how="left", left_on="ProductNo", right_on="ProductNo").merge(customer, how="left", left_on="CustomerNo", right_on="CustomerNo")
merged_transaction

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt
...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena


In [11]:
# Send a request to get data from url
url = os.getenv("conversion_rate_url")


In [12]:
result_conversion_rate = requests.get(url).json()


In [13]:
conversion_rate = pd.DataFrame(result_conversion_rate)
conversion_rate = conversion_rate.drop(columns=['id'])
conversion_rate['date'] = pd.to_datetime(conversion_rate['date'])

conversion_rate

,date,gbp_thb
0,2023-05-01,42.761
1,2023-05-02,42.477
2,2023-05-03,42.630
3,2023-05-04,42.456
4,2023-05-05,42.794
...,...,...
384,2024-05-19,45.957
385,2024-05-20,45.859
386,2024-05-21,46.246
387,2024-05-22,46.382


In [14]:
# merge 2 DataFrame (merged_transaction and conversion_rate)
final_df = merged_transaction.merge(conversion_rate, how="left", left_on="Date", right_on="date")

# create total_amount and thb_amount columns
final_df["total_amount"] = final_df["Price"] * final_df["Quantity"]
final_df["thb_amount"] = final_df["total_amount"] * final_df["gbp_thb"]

# drop unused columns and rename columns
final_df = final_df.drop(["date", "gbp_thb"], axis=1)
final_df.columns = ['transaction_id', 'date', 'product_id', 'price', 'quantity', 'customer_id',
                    'product_name', 'customer_country', 'customer_name', 'total_amount','thb_amount']

final_df

,transaction_id,date,product_id,price,quantity,customer_id,product_name,customer_country,customer_name,total_amount,thb_amount
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,257.64,11792.1828
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,383.40,17548.2180
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,138.36,6332.7372
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,127.80,5849.4060
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,71.64,3278.9628
...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,-37.92,-1616.5296
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,-42.27,-1801.9701
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,-11.74,-500.4762
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,-65.40,-2788.0020


In [15]:
# save file to parquet or csv

# final_df.to_parquet("output.parquet", index = False)
# final_df.to_csv('output.csv' ,index = False)